# Based on Drabas & Lee  -- Learning PySpark

## Resilient Distributed Datasets

### Creating RDDs

In [1]:
# set the kernel to python 2!
# you only need to run this cell if the spark context is not available when you start the notebook

import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
sc = SparkContext()

There are two ways to create an RDD in PySpark. 1) You can parallelize a list

In [2]:
data = sc.parallelize(
    [('Amber', 22), ('Alfred', 23), ('Skye',4), ('Albert', 12), 
     ('Amber', 9)])

In [3]:
print(data)

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:475


or 2) read from a repository (a file or a database)

In [4]:
!pwd

/home/ubuntu/Projects/PySparkDemos


In [5]:
data_from_file = sc.textFile('data/VS14MORT.txt.gz', 4)

In [6]:
print(data_from_file)

data/VS14MORT.txt.gz MapPartitionsRDD[2] at textFile at NativeMethodAccessorImpl.java:-2


The last parameter in sc.textFile(..., n) specifies the number of partitions the dataset is divided into.

Note, that to execute the code above you will have to change the path where the data is stored. The dataset can be downloaded from http://tomdrabas.com/data/VS14MORT.txt.gz

Spark can read from a multitude of filesystems: Local ones such as NTFS, FAT, or Mac OS Extended (HFS+), or distributed filesystems such as HDFS, S3, Cassandra, among many others

Note that Spark can automatically work with compressed datasets (like the Gzipped one in our preceding example).

Depending on how the data is read, the object holding it will be represented slightly  differently. The data read from a file is represented as MapPartitionsRDD instead  of ParallelCollectionRDD when we .paralellize(...) a collection.

#### Schema

RDDs are schema-less data structures (unlike DataFrames, which we will discuss later). Thus, parallelizing a dataset, such as in the following code snippet, 
is perfectly fine with Spark when using RDDs. We can mix amost everything: tuple, dict, list, etc...

In [8]:
data_heterogenous = sc.parallelize([('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain','visited', 4504]])
print(data_heterogenous)

ParallelCollectionRDD[4] at parallelize at PythonRDD.scala:475


In [9]:
data_heterogenous_collected = data_heterogenous.collect() 
print(data_heterogenous_collected)

[('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain', 'visited', 4504]]


The .collect() method returns all the elements of the RDD to the driver where it is serialized as a list

You can access the data in the object as you would normally do in Python.

In [10]:
data_heterogenous_collected[1]['Porsche']

100000

In [11]:
data_heterogenous_collected[2]

['Spain', 'visited', 4504]

In [12]:
data_heterogenous_collected[0]

('Ferrari', 'fast')

#### Reading from files

When you read from a text file, each row from the file forms an element of an RDD.

In [13]:
data_from_file.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 8, localhost): org.apache.spark.SparkException: 
Error from python worker:
  Traceback (most recent call last):
    File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 183, in _run_module_as_main
      mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
    File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 109, in _get_module_details
      __import__(pkg_name)
    File "<frozen importlib._bootstrap>", line 971, in _find_and_load
    File "<frozen importlib._bootstrap>", line 955, in _find_and_load_unlocked
    File "<frozen importlib._bootstrap>", line 656, in _load_unlocked
    File "<frozen importlib._bootstrap>", line 626, in _load_backward_compatible
    File "/home/ubuntu/spark/spark-2.0.0-bin-hadoop2.7/

In [10]:
data_from_file.take(2)

[u'                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601',
 u'                   1                                          2101  M1058 371708  4D3                2014U7CN                                    I250214 062   21 0311I250 61I272 62E669                                                                                                                                                            03 I250 E669 I272                                                                                       01  11                                 100 601']

#### User defined functions

You can create *longer* method to transform your data instead of using the `lambda` expression.

In [11]:
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]

    '''
        Input record schema
        schema: n-m (o) -- xxx
            n - position from
            m - position to
            o - number of characters
            xxx - description
        1. 1-19 (19) -- reserved positions
        2. 20 (1) -- resident status
        3. 21-60 (40) -- reserved positions
        4. 61-62 (2) -- education code (1989 revision)
        5. 63 (1) -- education code (2003 revision)
        6. 64 (1) -- education reporting flag
        7. 65-66 (2) -- month of death
        8. 67-68 (2) -- reserved positions
        9. 69 (1) -- sex
        10. 70 (1) -- age: 1-years, 2-months, 4-days, 5-hours, 6-minutes, 9-not stated
        11. 71-73 (3) -- number of units (years, months etc)
        12. 74 (1) -- age substitution flag (if the age reported in positions 70-74 is calculated using dates of birth and death)
        13. 75-76 (2) -- age recoded into 52 categories
        14. 77-78 (2) -- age recoded into 27 categories
        15. 79-80 (2) -- age recoded into 12 categories
        16. 81-82 (2) -- infant age recoded into 22 categories
        17. 83 (1) -- place of death
        18. 84 (1) -- marital status
        19. 85 (1) -- day of the week of death
        20. 86-101 (16) -- reserved positions
        21. 102-105 (4) -- current year
        22. 106 (1) -- injury at work
        23. 107 (1) -- manner of death
        24. 108 (1) -- manner of disposition
        25. 109 (1) -- autopsy
        26. 110-143 (34) -- reserved positions
        27. 144 (1) -- activity code
        28. 145 (1) -- place of injury
        29. 146-149 (4) -- ICD code
        30. 150-152 (3) -- 358 cause recode
        31. 153 (1) -- reserved position
        32. 154-156 (3) -- 113 cause recode
        33. 157-159 (3) -- 130 infant cause recode
        34. 160-161 (2) -- 39 cause recode
        35. 162 (1) -- reserved position
        36. 163-164 (2) -- number of entity-axis conditions
        37-56. 165-304 (140) -- list of up to 20 conditions
        57. 305-340 (36) -- reserved positions
        58. 341-342 (2) -- number of record axis conditions
        59. 343 (1) -- reserved position
        60-79. 344-443 (100) -- record axis conditions
        80. 444 (1) -- reserve position
        81. 445-446 (2) -- race
        82. 447 (1) -- bridged race flag
        83. 448 (1) -- race imputation flag
        84. 449 (1) -- race recode (3 categories)
        85. 450 (1) -- race recode (5 categories)
        86. 461-483 (33) -- reserved positions
        87. 484-486 (3) -- Hispanic origin
        88. 487 (1) -- reserved
        89. 488 (1) -- Hispanic origin/race recode
     '''

    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    
    #parsing starts. When parsing fails we just put a -99 there to indicated parsing failed in that row. 
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs
#     return record_split.split(row)

Note: Defining pure python methods can slow down your application because Spark constantly needs to switch between Python interpreter and JVM. Whenver possible, we should you built-in python functions. 

Now, instead of using `lambda` we will use the `extractInformation(...)` method to split and convert our dataset.

In [12]:
data_from_file_conv = data_from_file.map(extractInformation)

In [13]:
data_from_file_conv.map(lambda row: row).take(1)

[array([u'1', u'  ', u'2', u'1', u'01', u'M', u'1', u'087', u' ', u'43',
        u'23', u'11', u'  ', u'4', u'M', u'4', u'2014', u'U', u'7', u'C',
        u'N', u' ', u' ', u'I64 ', u'238', u'070', u'   ', u'24', u'01',
        u'11I64  ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ', u'01',
        u'I64  ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'01', u' ', u' ', u'1', u'1', u'100', u'6'],
       dtype='<U40')]

### Transformations

#### .map(...)

The method is applied to each element of the RDD: in the case for the `data_from_file_conv` dataset you can think of this as a transformation of each row.

In [14]:
data_2014 = data_from_file_conv.map(lambda row: int(row[16]))
data_2014.take(10)

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, -99]

You can combine more columns.

In [15]:
data_2014_2 = data_from_file_conv.map(lambda row: (row[16], int(row[16])))
data_2014_2.take(10)

[(u'2014', 2014),
 (u'2014', 2014),
 (u'2014', 2014),
 (u'2014', 2014),
 (u'2014', 2014),
 (u'2014', 2014),
 (u'2014', 2014),
 (u'2014', 2014),
 (u'2014', 2014),
 ('-99', -99)]

#### .filter(...)

The `.filter(...)` method allows you to select elements of your dataset that fit specified criteria.

In [16]:
data_filtered = data_from_file_conv.filter(lambda row: row[5] == 'F' and row[21] == '0')
data_filtered.count()

6

In [17]:
data_filtered

PythonRDD[10] at RDD at PythonRDD.scala:48

In [18]:
data_filtered.collect()

[array([u'2', u'12', u' ', u'0', u'07', u'F', u'1', u'030', u' ', u'32',
        u'12', u'05', u'  ', u'1', u'D', u'6', u'2014', u'N', u'1', u'U',
        u'Y', u'0', u'9', u'X44 ', u'420', u'122', u'   ', u'39', u'05',
        u'11T391 ', u'12X44  ', u'13T401 ', u'14T424 ', u'61F199 ',
        u'       ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ', u'05',
        u'X44  ', u'F199 ', u'T391 ', u'T401 ', u'T424 ', u'     ',
        u'     ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'01', u' ', u' ', u'1', u'1', u'100', u'6'],
       dtype='<U40'),
 array([u'1', u'  ', u'5', u'1', u'08', u'F', u'1', u'051', u' ', u'36',
        u'16', u'07', u'  ', u'4', u'D', u'4', u'2014', u'N', u'7', u'C',
        u'N', u'0', u'0', u'C159', u'077', u'021', u' 

#### .flatMap(...)

The `.flatMap(...)` method works similarly to `.map(...)` but returns a flattened results instead of a list. 

In [19]:
data_filtered_flat = data_filtered.flatMap(lambda row: (row[16], int(row[16]) + 1))

In [20]:
data_filtered_flat.count()

12

In [21]:
data_filtered_flat.collect()

[u'2014',
 2015,
 u'2014',
 2015,
 u'2014',
 2015,
 u'2014',
 2015,
 u'2014',
 2015,
 u'2014',
 2015]

In [22]:
data_2014_flat = data_from_file_conv.flatMap(lambda row: (row[16], int(row[16]) + 1))
data_2014_flat.take(10)

[u'2014', 2015, u'2014', 2015, u'2014', 2015, u'2014', 2015, u'2014', 2015]

In [23]:
data_2014_flat.count()

5262342

#### .distinct()

This method returns a list of distinct values in a specified column.

In [24]:
distinct_gender = data_from_file_conv.map(lambda row: row[5]).distinct().collect()
distinct_gender

['-99', u'M', u'F']

#### .sample(...)

The `.sample()` method returns a randomized sample from the dataset.

In [25]:
fraction = 0.01
#False, fraction, 666 = With raplecement? Fraction of dataset used to sampling, random seed
data_sample = data_from_file_conv.sample(False, fraction, 666)

data_sample.take(1)

[array([u'1', u'  ', u'3', u'1', u'01', u'F', u'1', u'045', u' ', u'35',
        u'15', u'07', u'  ', u'4', u'M', u'6', u'2014', u'U', u'7', u'C',
        u'N', u' ', u' ', u'I48 ', u'228', u'068', u'   ', u'22', u'01',
        u'11I48  ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ', u'01',
        u'I48  ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'01', u' ', u' ', u'1', u'1', u'100', u'6'],
       dtype='<U40')]

Let's confirm that we really got 1% of all the records.

In [26]:
print('Original dataset: {0}, sample: {1}'.format(data_from_file_conv.count(), data_sample.count()))

Original dataset: 2631171, sample: 26242


#### .leftOuterJoin(...)

Left outer join, just like the SQL world, joins two RDDs based on the values found in both datasets, and returns records from the left RDD with records from the right one appended where the two RDDs match.

In [27]:
rdd1 = sc.parallelize([('a', 1), ('b', 4), ('c',10)])
rdd2 = sc.parallelize([('a', 4), ('a', 1), ('b', '6'), ('d', 15)])

rdd3 = rdd1.leftOuterJoin(rdd2)
rdd3.take(5)

[('a', (1, 4)), ('a', (1, 1)), ('c', (10, None)), ('b', (4, '6'))]

d is missing since this is only a leftOuterJoin

If we used `.join(...)` method instead we would have gotten only the values for `'a'` and `'b'` as these two values intersect between these two RDDs.

In [28]:
rdd4 = rdd1.join(rdd2)
rdd4.collect()

[('a', (1, 4)), ('a', (1, 1)), ('b', (4, '6'))]

Another useful method is the `.intersection(...)` that returns the records that are *equal* in both RDDs.

In [29]:
rdd5 = rdd1.intersection(rdd2)
rdd5.collect()

[('a', 1)]

#### .repartition(...)

Repartitioning the dataset changes the number of partitions the dataset is divided into.

In [30]:
rdd1 = rdd1.repartition(4)

len(rdd1.glom().collect())

4

In [31]:
rdd1.glom().collect()

[[], [], [], [('a', 1), ('b', 4), ('c', 10)]]

### Actions

#### .take(...)

The method returns `n` top rows from a single data partition.

In [32]:
data_first = data_from_file_conv.take(1)
data_first

[array([u'1', u'  ', u'2', u'1', u'01', u'M', u'1', u'087', u' ', u'43',
        u'23', u'11', u'  ', u'4', u'M', u'4', u'2014', u'U', u'7', u'C',
        u'N', u' ', u' ', u'I64 ', u'238', u'070', u'   ', u'24', u'01',
        u'11I64  ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ', u'01',
        u'I64  ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'01', u' ', u' ', u'1', u'1', u'100', u'6'],
       dtype='<U40')]

If you want somewhat randomized records you can use `.takeSample(...)` instead.

In [33]:
data_take_sampled = data_from_file_conv.takeSample(False, 2, 667)
data_take_sampled

[array([u'1', u'  ', u'3', u'1', u'01', u'F', u'1', u'085', u' ', u'43',
        u'23', u'11', u'  ', u'4', u'D', u'6', u'2014', u'U', u'7', u'B',
        u'N', u' ', u' ', u'I251', u'215', u'063', u'   ', u'21', u'05',
        u'11I259 ', u'21I251 ', u'61I500 ', u'62E149 ', u'63F179 ',
        u'       ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ',
        u'       ', u'       ', u'       ', u'       ', u'       ', u'04',
        u'I251 ', u'E149 ', u'F179 ', u'I500 ', u'     ', u'     ',
        u'     ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'     ', u'     ', u'     ', u'     ',
        u'     ', u'     ', u'01', u' ', u' ', u'1', u'1', u'100', u'6'],
       dtype='<U40'),
 array([u'1', u'  ', u'3', u'1', u'10', u'F', u'1', u'093', u' ', u'44',
        u'24', u'11', u'  ', u'4', u'W', u'7', u'2014', u'U', u'7', u'B',
        u'N', u' ', u' ', u'E875', u'172', u'111', u' 

#### .reduce(...)

Another action that processes your data, the `.reduce(...)` method *reduces* the elements of an RDD using a specified method.

In [34]:
rdd1.collect()

[('a', 1), ('b', 4), ('c', 10)]

In [35]:
rdd1.map(lambda row: row[1]).reduce(lambda x, y: x + y)

15

If the reducing function is not associative and commutative you will sometimes get wrong results depending how your data is partitioned.

In [36]:
rdd1.map(lambda row: row[1]).reduce(lambda x, y: x / y)

0

In [37]:
data_reduce = sc.parallelize([1.0, 2.0, .5, .1, 5, .2], 1)

In [38]:
data_reduce.collect()

[1.0, 2.0, 0.5, 0.1, 5, 0.2]

If we were to reduce the data in a manner that we would like to *divide* the current result by the subsequent one, we would expect a value of 10

In [39]:
works = data_reduce.reduce(lambda x, y: x / y)
works

10.0

However, if you were to partition the data into 3 partitions, the result will be wrong.

In [40]:
data_reduce = sc.parallelize([1.0, 2.0, .5, .1, 5, .2], 3)
data_reduce.reduce(lambda x, y: x / y)

0.004

The `.reduceByKey(...)` method works in a similar way to the `.reduce(...)` method but performs a reduction on a key-by-key basis.

In [41]:
data_key = sc.parallelize([('a', 4),('b', 3),('c', 2),('a', 8),('d', 2),('b', 1),('d', 3)],4)
data_key.reduceByKey(lambda x, y: x + y).collect()

[('a', 12), ('d', 5), ('c', 2), ('b', 4)]

#### .count()

The `.count()` method counts the number of elements in the RDD.

In [42]:
data_reduce.count()

6

It has the same effect as the method below but does not require shifting the data to the driver.

In [43]:
len(data_reduce.collect()) # WRONG -- DON'T DO THIS!

6

If your dataset is in a form of a *key-value* you can use the `.countByKey()` method to get the counts of distinct keys.

In [44]:
data_key.countByKey().items()

[('a', 2), ('c', 1), ('b', 2), ('d', 2)]

#### .saveAsTextFile(...)

As the name suggests, the `.saveAsTextFile()` the RDD and saves it to text files: each partition to a separate file.

In [47]:
data_key.saveAsTextFile('data_key.txt')

To read it back, you need to parse it back as, as before, all the rows are treated as strings.

In [48]:
def parseInput(row):
    import re
    
    pattern = re.compile(r'\(\'([a-z])\', ([0-9])\)')
    row_split = pattern.split(row)
    
    return (row_split[1], int(row_split[2]))
    
data_key_reread = sc \
    .textFile('data_key.txt') \
    .map(parseInput)
data_key_reread.collect()

[(u'b', 3), (u'c', 2), (u'a', 4), (u'b', 1), (u'd', 3), (u'a', 8), (u'd', 2)]

**.foreach(...)**

A method that applies the same function to each element of the RDD in an iterative way.

In [49]:
# this prints to terminal not to jupyter notebook!
def f(x): 
    print(x)
    print('hi')

data_key.foreach(f)

In [50]:
data_key.collect()

[('a', 4), ('b', 3), ('c', 2), ('a', 8), ('d', 2), ('b', 1), ('d', 3)]